In [1]:
import os
import sys
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ml.models import c3d

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
import os
import csv
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from google.cloud import storage

BLACKLIST = []


class ChunkGenerator(object):
    def __init__(self, dims=(32, 32, 32), batch_size=16,
                 shuffle=True,
                 validation=False,
                 test=False, split_test=False,
                 split=0.2, extend_dims=True,
                 augment_data=True):
        self.dims = dims
        self.batch_size = batch_size
        self.extend_dims = extend_dims
        self.augment_data = augment_data
        self.validation = validation

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=[1.0, 1.1],
        )

        # Access Google Cloud Storage
        gcs_client = storage.Client.from_service_account_json(
            '/home/harold_triedman/elvo-analysis/credentials/client_secret.json'
        )
        bucket = gcs_client.get_bucket('elvos')

        # Get label data from Google Cloud Storage
        blob = storage.Blob('annotated_labels.csv', bucket)
        print('hi')
        blob.download_to_filename('tmp/augmented_annotated_labels.csv')
        prelim_label_data = {}
        with open('tmp/augmented_annotated_labels.csv', 'r') as pos_file:
            reader = csv.reader(pos_file, delimiter=',')
            for row in reader:
                if row[1] != 'label':
                    prelim_label_data[row[0]] = int(row[1])

        # Get 12096 random negatives from the label data to feed into our generator
        negative_counter = 0
        negative_label_data = {}
        print("getting 12096 random negative labels")
        while negative_counter < 12096:
            id_, label = random.choice(list(prelim_label_data.items()))
            if label == 0:
                negative_label_data[id_] = label
                del prelim_label_data[id_]
                negative_counter += 1

        print(list(negative_label_data.items()))
                
        # Get all of the positives from the label data
        label_data = {}
        print('getting positive labels')
        for id_, label in list(prelim_label_data.items()):
            if label == 1:
                for i in range(1, 25):
                    label_data[id_ + '_' + str(i)] = label

        print(list(label_data.items()))
        
        # Put pos and neg together into one dictionary
        label_data.update(negative_label_data)

        # Get positive chunks
        print('getting positive chunks')
        pos_blobs = bucket.list_blobs(prefix='chunk_data/normal/positive')
        files = []
        for blob in pos_blobs:
            file = blob.name
            file = file.split('/')[-1]
            file = file.split('.')[0]
            print(file)

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            if not blacklisted:
                # Add all data augmentation methods
                files.append({
                    "name": file,
                })

        print("getting negative chunks")
        # Get negative chunks that were chosen to be in the labels
        neg_blobs = bucket.list_blobs(prefix='chunk_data/normal/negative')
        for blob in neg_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if file_id in negative_label_data and not blacklisted:
                files.append({
                    "name": file,
                })

        # convert labels from dict to np array
        print('convert labels from dict to np array')
        labels = np.zeros(len(files))
        for i, file in enumerate(files):
            filename = file['name']
            filename = filename.split('/')[-1]
            filename = filename.split('.')[0]
            print(filename)
            labels[i] = label_data[filename]

        # Take into account shuffling
        if shuffle:
            tmp = list(zip(files, labels))
            random.shuffle(tmp)
            files, labels = zip(*tmp)
            labels = np.array(labels)

        # Split based on validation
        print('splitting based on validation split')
        if validation:
            if split_test:
                files = files[:int(len(files) * split / 2)]
                labels = labels[:int(len(labels) * split / 2)]
            else:
                files = files[:int(len(files) * split)]
                labels = labels[:int(len(labels) * split)]
        elif test:
            if split_test:
                files = files[int(len(files) * split / 2):
                              int(len(files) * split)]
                labels = labels[int(len(labels) * split / 2):
                                int(len(labels) * split)]
            else:
                raise ValueError('must set split_test to True if test')
        else:
            files = files[int(len(files) * split):]
            labels = labels[int(len(labels) * split):]
        print(np.shape(files))
        print(np.shape(labels))
        print("Negatives: {}".format(np.count_nonzero(labels == 0)))
        print("Positives: {}".format(np.count_nonzero(labels)))
        self.files = files
        self.labels = labels
        self.bucket = bucket

    def generate(self):
        steps = self.get_steps_per_epoch()
        # print(steps)
        while True:
            for i in range(steps):
                # print(i)
                # print("D")
                x, y = self.__data_generation(i)
                yield x, y

    def get_steps_per_epoch(self):
        return len(self.files) // self.batch_size

    def __data_generation(self, i):
        bsz = self.batch_size
        files = self.files[i * bsz:(i + 1) * bsz]
        labels = self.labels[i * bsz:(i + 1) * bsz]
        images = []

        # Download files to tmp/npy/
        for i, file in enumerate(files):
            file_id = file['name'].split('/')[-1]
            file_id = file_id.split('.')[0]
            print(file_id)
            img = np.load('tmp/npy/{}.npy'.format(file_id))
            # print(np.shape(img))
            images.append(img)
        images = np.array(images)
        # print("Loaded entire batch.")
        # print(np.shape(images))
        return images, labels



In [19]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
TrainGen = ChunkGenerator()
ValGen = ChunkGenerator()
train_gen = TrainGen(dims=(32, 32, 32), # dimensions may
                                         # be different
                                         # depending on what
                                         # data we're using
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = ValGen(dims=(32, 32, 32),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

hi
getting 12096 random negative labels
[('GFRJ7SPGIEJVXVRS907', 0), ('DXLPP6QTNOHQRHVG203', 0), ('TFV4472GKPYD269N142', 0), ('PBR2STY62FZ3MQHZ555', 0), ('TVY4XBCLXJBKK72S522', 0), ('ZTFA5BJQHNJIRH4T507', 0), ('Q8LO6RYLTPQQKJO8278', 0), ('JOJU5LPDDBBP6LYG309', 0), ('KOM02ZTT5IZQ41QL456', 0), ('BIASJO43N3A7Z63W113', 0), ('RWKA32WBSVFB4MQF125', 0), ('RTC5RXQHUXCSPNLN266', 0), ('LRCPYDPZFFC6HR6K595', 0), ('HO0Y1BBUIJE9IJIJ524', 0), ('PQJEIXKIJMZTBZ74760', 0), ('BMBQ1TUZFOM1YU3Y358', 0), ('BGK2KBB8KXRWI65O1728', 0), ('C4X729LF9L0LXQ1D28', 0), ('ILNTKMBVTXNXURGV162', 0), ('RLJSRNLTRUQAVT3L127', 0), ('IIOOPK6T2ZWDZMU3143', 0), ('NYKZ3FK2LQ1Q5UP3412', 0), ('OMC7W51BOE9ILIWF164', 0), ('0VU79X3RK3L2GY8G184', 0), ('5LGDWR05UU6NKX4Z210', 0), ('DVYKX4XMBHD57MZJ288', 0), ('KEKAAUIJ2RZ7QYC6315', 0), ('N87RZI5DMKV4CP2C299', 0), ('BSXCZXMR5T9ANRE91', 0), ('CCGEZOFGLKC3USJ8905', 0), ('KSPGRITHL4N0XPLW515', 0), ('POSX6PS6LR34HENB694', 0), ('BRSZTKDWTB2X5P3W579', 0), ('0EZJW4R4EMC16I1040', 0), ('ZMEGS4G4

04IOS24JP70LHBGB184_1
04IOS24JP70LHBGB184_10
04IOS24JP70LHBGB184_11
04IOS24JP70LHBGB184_12
04IOS24JP70LHBGB184_13
04IOS24JP70LHBGB184_14
04IOS24JP70LHBGB184_15
04IOS24JP70LHBGB184_16
04IOS24JP70LHBGB184_17
04IOS24JP70LHBGB184_18
04IOS24JP70LHBGB184_19
04IOS24JP70LHBGB184_2
04IOS24JP70LHBGB184_20
04IOS24JP70LHBGB184_21
04IOS24JP70LHBGB184_22
04IOS24JP70LHBGB184_23
04IOS24JP70LHBGB184_24
04IOS24JP70LHBGB184_3
04IOS24JP70LHBGB184_4
04IOS24JP70LHBGB184_5
04IOS24JP70LHBGB184_6
04IOS24JP70LHBGB184_7
04IOS24JP70LHBGB184_8
04IOS24JP70LHBGB184_9
0DQO9A6UXUQHR8RA148_1
0DQO9A6UXUQHR8RA148_10
0DQO9A6UXUQHR8RA148_11
0DQO9A6UXUQHR8RA148_12
0DQO9A6UXUQHR8RA148_13
0DQO9A6UXUQHR8RA148_14
0DQO9A6UXUQHR8RA148_15
0DQO9A6UXUQHR8RA148_16
0DQO9A6UXUQHR8RA148_17
0DQO9A6UXUQHR8RA148_18
0DQO9A6UXUQHR8RA148_19
0DQO9A6UXUQHR8RA148_2
0DQO9A6UXUQHR8RA148_20
0DQO9A6UXUQHR8RA148_21
0DQO9A6UXUQHR8RA148_22
0DQO9A6UXUQHR8RA148_23
0DQO9A6UXUQHR8RA148_24
0DQO9A6UXUQHR8RA148_3
0DQO9A6UXUQHR8RA148_4
0DQO9A6UXUQHR8RA148_5
0D

48XO6UQZI766F8TN436_24
48XO6UQZI766F8TN436_3
48XO6UQZI766F8TN436_4
48XO6UQZI766F8TN436_5
48XO6UQZI766F8TN436_6
48XO6UQZI766F8TN436_7
48XO6UQZI766F8TN436_8
48XO6UQZI766F8TN436_9
4DIUJ5A6193W879P114_1
4DIUJ5A6193W879P114_10
4DIUJ5A6193W879P114_11
4DIUJ5A6193W879P114_12
4DIUJ5A6193W879P114_13
4DIUJ5A6193W879P114_14
4DIUJ5A6193W879P114_15
4DIUJ5A6193W879P114_16
4DIUJ5A6193W879P114_17
4DIUJ5A6193W879P114_18
4DIUJ5A6193W879P114_19
4DIUJ5A6193W879P114_2
4DIUJ5A6193W879P114_20
4DIUJ5A6193W879P114_21
4DIUJ5A6193W879P114_22
4DIUJ5A6193W879P114_23
4DIUJ5A6193W879P114_24
4DIUJ5A6193W879P114_3
4DIUJ5A6193W879P114_4
4DIUJ5A6193W879P114_5
4DIUJ5A6193W879P114_6
4DIUJ5A6193W879P114_7
4DIUJ5A6193W879P114_8
4DIUJ5A6193W879P114_9
4EH0PS1IKFIGT1T0114_1
4EH0PS1IKFIGT1T0114_10
4EH0PS1IKFIGT1T0114_11
4EH0PS1IKFIGT1T0114_12
4EH0PS1IKFIGT1T0114_13
4EH0PS1IKFIGT1T0114_14
4EH0PS1IKFIGT1T0114_15
4EH0PS1IKFIGT1T0114_16
4EH0PS1IKFIGT1T0114_17
4EH0PS1IKFIGT1T0114_18
4EH0PS1IKFIGT1T0114_19
4EH0PS1IKFIGT1T0114_2
4EH0PS

8KZMN84B4R5RPP2Q245_17
8KZMN84B4R5RPP2Q245_18
8KZMN84B4R5RPP2Q245_19
8KZMN84B4R5RPP2Q245_2
8KZMN84B4R5RPP2Q245_20
8KZMN84B4R5RPP2Q245_21
8KZMN84B4R5RPP2Q245_22
8KZMN84B4R5RPP2Q245_23
8KZMN84B4R5RPP2Q245_24
8KZMN84B4R5RPP2Q245_3
8KZMN84B4R5RPP2Q245_4
8KZMN84B4R5RPP2Q245_5
8KZMN84B4R5RPP2Q245_6
8KZMN84B4R5RPP2Q245_7
8KZMN84B4R5RPP2Q245_8
8KZMN84B4R5RPP2Q245_9
8LL3IE6PJ9GC2JX0170_1
8LL3IE6PJ9GC2JX0170_10
8LL3IE6PJ9GC2JX0170_11
8LL3IE6PJ9GC2JX0170_12
8LL3IE6PJ9GC2JX0170_13
8LL3IE6PJ9GC2JX0170_14
8LL3IE6PJ9GC2JX0170_15
8LL3IE6PJ9GC2JX0170_16
8LL3IE6PJ9GC2JX0170_17
8LL3IE6PJ9GC2JX0170_18
8LL3IE6PJ9GC2JX0170_19
8LL3IE6PJ9GC2JX0170_2
8LL3IE6PJ9GC2JX0170_20
8LL3IE6PJ9GC2JX0170_21
8LL3IE6PJ9GC2JX0170_22
8LL3IE6PJ9GC2JX0170_23
8LL3IE6PJ9GC2JX0170_24
8LL3IE6PJ9GC2JX0170_3
8LL3IE6PJ9GC2JX0170_4
8LL3IE6PJ9GC2JX0170_5
8LL3IE6PJ9GC2JX0170_6
8LL3IE6PJ9GC2JX0170_7
8LL3IE6PJ9GC2JX0170_8
8LL3IE6PJ9GC2JX0170_9
8SREEDHBQF4ZGA02165_1
8SREEDHBQF4ZGA02165_10
8SREEDHBQF4ZGA02165_11
8SREEDHBQF4ZGA02165_12
8SREED

C02TH0J1FT7XQ5EU268_1
C02TH0J1FT7XQ5EU268_10
C02TH0J1FT7XQ5EU268_11
C02TH0J1FT7XQ5EU268_12
C02TH0J1FT7XQ5EU268_13
C02TH0J1FT7XQ5EU268_14
C02TH0J1FT7XQ5EU268_15
C02TH0J1FT7XQ5EU268_16
C02TH0J1FT7XQ5EU268_17
C02TH0J1FT7XQ5EU268_18
C02TH0J1FT7XQ5EU268_19
C02TH0J1FT7XQ5EU268_2
C02TH0J1FT7XQ5EU268_20
C02TH0J1FT7XQ5EU268_21
C02TH0J1FT7XQ5EU268_22
C02TH0J1FT7XQ5EU268_23
C02TH0J1FT7XQ5EU268_24
C02TH0J1FT7XQ5EU268_3
C02TH0J1FT7XQ5EU268_4
C02TH0J1FT7XQ5EU268_5
C02TH0J1FT7XQ5EU268_6
C02TH0J1FT7XQ5EU268_7
C02TH0J1FT7XQ5EU268_8
C02TH0J1FT7XQ5EU268_9
C0BWKG4H07GUBILW129_1
C0BWKG4H07GUBILW129_10
C0BWKG4H07GUBILW129_11
C0BWKG4H07GUBILW129_12
C0BWKG4H07GUBILW129_13
C0BWKG4H07GUBILW129_14
C0BWKG4H07GUBILW129_15
C0BWKG4H07GUBILW129_16
C0BWKG4H07GUBILW129_17
C0BWKG4H07GUBILW129_18
C0BWKG4H07GUBILW129_19
C0BWKG4H07GUBILW129_2
C0BWKG4H07GUBILW129_20
C0BWKG4H07GUBILW129_21
C0BWKG4H07GUBILW129_22
C0BWKG4H07GUBILW129_23
C0BWKG4H07GUBILW129_24
C0BWKG4H07GUBILW129_3
C0BWKG4H07GUBILW129_4
C0BWKG4H07GUBILW129_5
C0

...

CSCIKXOMNAIB3LUQ292
CSCIKXOMNAIB3LUQ31
CSCIKXOMNAIB3LUQ357
CSCIKXOMNAIB3LUQ420
CSCIKXOMNAIB3LUQ45
CSFRT7ES11I1Y3CF101
CSFRT7ES11I1Y3CF154
CSFRT7ES11I1Y3CF166
CSFRT7ES11I1Y3CF23
CSFRT7ES11I1Y3CF332
CSFRT7ES11I1Y3CF34
CSFRT7ES11I1Y3CF348
CSFRT7ES11I1Y3CF361
CSFRT7ES11I1Y3CF362
CSFRT7ES11I1Y3CF388
CSFRT7ES11I1Y3CF397
CSFRT7ES11I1Y3CF443
CSFRT7ES11I1Y3CF547
CSFRT7ES11I1Y3CF56
CSFRT7ES11I1Y3CF562
CSFRT7ES11I1Y3CF574
CSFRT7ES11I1Y3CF618
CSFRT7ES11I1Y3CF91
CSFRT7ES11I1Y3CF97
CSFRT7ES11I1Y3CF98
CSQRNTS8K61O7WP5252
CSQRNTS8K61O7WP5315
CSQRNTS8K61O7WP5413
CSQRNTS8K61O7WP5486
CSQRNTS8K61O7WP5684
CSQRNTS8K61O7WP574
CSXRYWJCXJXAJRLP152
CSXRYWJCXJXAJRLP162
CSXRYWJCXJXAJRLP232
CSXRYWJCXJXAJRLP24
CSXRYWJCXJXAJRLP326
CSXRYWJCXJXAJRLP333
CSXRYWJCXJXAJRLP391
CSXRYWJCXJXAJRLP574
CSXRYWJCXJXAJRLP575
CSXRYWJCXJXAJRLP594
CSXRYWJCXJXAJRLP63
CUM2CZJVM5H5ZKFE158
CUM2CZJVM5H5ZKFE196
CUM2CZJVM5H5ZKFE217
CUM2CZJVM5H5ZKFE282
CUM2CZJVM5H5ZKFE426
CUM2CZJVM5H5ZKFE48
CUM2CZJVM5H5ZKFE671
CUM2CZJVM5H5ZKFE68
CUM2CZJVM5H5Z

J3CQA68PDJ31TV8R246
J3CQA68PDJ31TV8R267
J3CQA68PDJ31TV8R269
J3CQA68PDJ31TV8R273
J3CQA68PDJ31TV8R334
J3CQA68PDJ31TV8R354
J3CQA68PDJ31TV8R371
J3CQA68PDJ31TV8R44
J3CQA68PDJ31TV8R472
J3CQA68PDJ31TV8R61
J3CQA68PDJ31TV8R94
J3CQA68PDJ31TV8R964
J3JHPC3E15NZGX34126
J3JHPC3E15NZGX34157
J3JHPC3E15NZGX34195
J3JHPC3E15NZGX34299
J3JHPC3E15NZGX3430
J3JHPC3E15NZGX34341
J3JHPC3E15NZGX3439
J3JHPC3E15NZGX3445
J3JHPC3E15NZGX34479
J3JHPC3E15NZGX3454
J3JHPC3E15NZGX3457
J3JHPC3E15NZGX34611
J3JHPC3E15NZGX3484
J3JHPC3E15NZGX3489
J3N2GD4RC25OFS0Y122
J3N2GD4RC25OFS0Y141
J3N2GD4RC25OFS0Y18
J3N2GD4RC25OFS0Y180
J3N2GD4RC25OFS0Y305
J3N2GD4RC25OFS0Y341
J3N2GD4RC25OFS0Y36
J3N2GD4RC25OFS0Y463
J3N2GD4RC25OFS0Y48
J3N2GD4RC25OFS0Y9
JAANEAKBEZPKV4JV105
JAANEAKBEZPKV4JV115
JAANEAKBEZPKV4JV122
JAANEAKBEZPKV4JV128
JAANEAKBEZPKV4JV163
JAANEAKBEZPKV4JV167
JAANEAKBEZPKV4JV173
JAANEAKBEZPKV4JV330
JAANEAKBEZPKV4JV336
JAANEAKBEZPKV4JV357
JAANEAKBEZPKV4JV471
JAANEAKBEZPKV4JV526
JAANEAKBEZPKV4JV54
JAS2TYJWKQL65CQT117
JAS2TYJWKQL65CQT

POYYSD3RSJFBKSLQ177
POYYSD3RSJFBKSLQ18
POYYSD3RSJFBKSLQ189
POYYSD3RSJFBKSLQ20
POYYSD3RSJFBKSLQ21
POYYSD3RSJFBKSLQ247
POYYSD3RSJFBKSLQ282
POYYSD3RSJFBKSLQ299
POYYSD3RSJFBKSLQ418
POYYSD3RSJFBKSLQ467
POYYSD3RSJFBKSLQ474
POYYSD3RSJFBKSLQ481
POYYSD3RSJFBKSLQ503
POYYSD3RSJFBKSLQ53
POYYSD3RSJFBKSLQ556
POYYSD3RSJFBKSLQ73
POYYSD3RSJFBKSLQ81
POYYSD3RSJFBKSLQ98
PQ918CDGQIFXXXN1122
PQ918CDGQIFXXXN1147
PQ918CDGQIFXXXN1150
PQ918CDGQIFXXXN1275
PQ918CDGQIFXXXN1346
PQ918CDGQIFXXXN1364
PQ918CDGQIFXXXN141
PQ918CDGQIFXXXN1462
PQ918CDGQIFXXXN1500
PQ918CDGQIFXXXN1516
PQ918CDGQIFXXXN1623
PQ918CDGQIFXXXN1750
PQ918CDGQIFXXXN1762
PQ918CDGQIFXXXN1840
PQJEIXKIJMZTBZ74115
PQJEIXKIJMZTBZ74196
PQJEIXKIJMZTBZ74206
PQJEIXKIJMZTBZ74226
PQJEIXKIJMZTBZ74265
PQJEIXKIJMZTBZ7429
PQJEIXKIJMZTBZ74329
PQJEIXKIJMZTBZ7457
PQJEIXKIJMZTBZ74596
PQJEIXKIJMZTBZ74622
PQJEIXKIJMZTBZ7464
PQJEIXKIJMZTBZ7465
PQJEIXKIJMZTBZ74704
PQJEIXKIJMZTBZ74760
PQLRGWB8W5XDIP1B138
PQLRGWB8W5XDIP1B177
PQLRGWB8W5XDIP1B193
PQLRGWB8W5XDIP1B249
PQLRGWB8W5XD

VXXCYRTUJK849I60916
VXXCYRTUJK849I60940
VXXCYRTUJK849I60985
VZHXKDEYF7EK4RRP126
VZHXKDEYF7EK4RRP138
VZHXKDEYF7EK4RRP288
VZHXKDEYF7EK4RRP32
VZHXKDEYF7EK4RRP37
VZHXKDEYF7EK4RRP456
VZHXKDEYF7EK4RRP490
VZHXKDEYF7EK4RRP525
VZHXKDEYF7EK4RRP545
VZHXKDEYF7EK4RRP59
VZHXKDEYF7EK4RRP680
VZHXKDEYF7EK4RRP780
VZOXMEUK7BN7PQH3135
VZOXMEUK7BN7PQH3189
VZOXMEUK7BN7PQH3257
VZOXMEUK7BN7PQH3309
VZOXMEUK7BN7PQH3444
VZOXMEUK7BN7PQH3507
VZOXMEUK7BN7PQH3551
VZOXMEUK7BN7PQH3713
VZOXMEUK7BN7PQH376
VZOXMEUK7BN7PQH391
VZOXMEUK7BN7PQH393
W098UN5Y50II02BX111
W098UN5Y50II02BX112
W098UN5Y50II02BX147
W098UN5Y50II02BX17
W098UN5Y50II02BX238
W098UN5Y50II02BX266
W098UN5Y50II02BX267
W098UN5Y50II02BX276
W098UN5Y50II02BX290
W098UN5Y50II02BX405
W098UN5Y50II02BX410
W098UN5Y50II02BX468
W098UN5Y50II02BX474
W098UN5Y50II02BX545
W098UN5Y50II02BX557
W098UN5Y50II02BX597
W098UN5Y50II02BX62
W26II23AJEMSREDL196
W26II23AJEMSREDL285
W26II23AJEMSREDL309
W26II23AJEMSREDL402
W26II23AJEMSREDL68
W6OUXVVFEFZSLKWT112
W6OUXVVFEFZSLKWT180
W6OUXVVFE

ZXXMMCGK6ANRKLFD231
ZXXMMCGK6ANRKLFD379
ZXXMMCGK6ANRKLFD437
ZXXMMCGK6ANRKLFD517
ZXXMMCGK6ANRKLFD52
ZXXMMCGK6ANRKLFD58
ZYKNNBEIY0387GUQ149
ZYKNNBEIY0387GUQ228
ZYKNNBEIY0387GUQ234
ZYKNNBEIY0387GUQ235
ZYKNNBEIY0387GUQ468
ZYKNNBEIY0387GUQ618
ZYKNNBEIY0387GUQ96
ZZWCWIY01XXP36WL11
ZZWCWIY01XXP36WL1248
ZZWCWIY01XXP36WL1260
ZZWCWIY01XXP36WL155
ZZWCWIY01XXP36WL188
ZZWCWIY01XXP36WL215
ZZWCWIY01XXP36WL224
ZZWCWIY01XXP36WL279
ZZWCWIY01XXP36WL283
ZZWCWIY01XXP36WL303
ZZWCWIY01XXP36WL317
ZZWCWIY01XXP36WL335
ZZWCWIY01XXP36WL528
ZZWCWIY01XXP36WL537
ZZWCWIY01XXP36WL545
ZZWCWIY01XXP36WL549
ZZWCWIY01XXP36WL57
ZZWCWIY01XXP36WL63
ZZWCWIY01XXP36WL65
ZZWCWIY01XXP36WL650
ZZWCWIY01XXP36WL652
ZZWCWIY01XXP36WL673
ZZWCWIY01XXP36WL884
ZZWCWIY01XXP36WL90
ZZWCWIY01XXP36WL915
ZZWCWIY01XXP36WL984
ZZX0ZNWG6Q9I18GK116
ZZX0ZNWG6Q9I18GK118
ZZX0ZNWG6Q9I18GK179
ZZX0ZNWG6Q9I18GK84
splitting based on validation split
(19354,)
(19354,)
Negatives: 9679
Positives: 9675
hi
getting 12096 random negative labels
